# Devices

In [1]:
import json
from eventbus import bus
from eventbus.devices import Device, State, Actuator, Transducer

bus.leaf_id = "test"

@bus.on("*")
def events(topic, **kw):
    if topic == '!state': #  or topic.startswith('?act'):
        print(f"EVENT {topic}:", kw)

## Device Registry

In [2]:
async def callback(device, uid, state=None, brightness=None):
    print(f"CB action for {device.uid()} ({device.attributes['name']}): uid={uid} on_off={state} brightness={brightness}")
    if state is not None: await device.update("state", state)
    if brightness is not None: await device.update("brightness", brightness) 

Device("light1", Transducer("state", callback), Transducer("brightness", callback), name="Christmas light")

print(dir(bus))
await bus.emit(topic="?act/test.light1.state", state='on')
await bus.emit(topic="?act/test.light1.brightness", brightness=100)

['__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_call_handler', '_sync_emit_task', 'emit', 'emit_event', 'emit_event_sync', 'emit_sync', 'event_queue', 'leaf_id', 'listeners', 'on', 'pause', 'subscribe', 'sync_queue_size', 'unsubscribe', 'unsubscribe_all']


## BinarySensor

In [3]:
from eventbus import BinarySensor

binary_sensor = BinarySensor("window")
print(binary_sensor)

await binary_sensor.update(True)

BinarySensor(window, State(test.window.state), {})
EVENT !state: {'uid': 'test.window.state', 'value': 'ON', 'timestamp': 1723506587.114561, 'src': 'test'}


## Sensor

In [4]:
from eventbus import Sensor, State

climate_sensor = Sensor("climate",
    State("temperature", unit="°C"),
    State("humidity", unit="%")
)

print(climate_sensor)

await climate_sensor.update("temperature", 25)
await climate_sensor.update("temperature", 25)  # no change: ignored
await climate_sensor.update("humidity", 66)
await climate_sensor.update("bogus", 0)         # not declared: ignored


Sensor(climate, State(test.climate.temperature), State(test.climate.humidity), {})
EVENT !state: {'uid': 'test.climate.temperature', 'value': 25, 'timestamp': 1723506587.114561, 'src': 'test'}
EVENT !state: {'uid': 'test.climate.temperature', 'value': 25, 'timestamp': 1723506587.114561, 'src': 'test'}
EVENT !state: {'uid': 'test.climate.humidity', 'value': 66, 'timestamp': 1723506587.114561, 'src': 'test'}
EVENT !state: {'uid': 'test.climate.bogus', 'value': 0, 'timestamp': 1723506587.114561, 'src': 'test'}


## Switch

In [5]:
from eventbus import bus, Switch

async def cb(switch, uid, value):
    # set switch to requested state, then update state
    await switch.update(value)

sw1 = Switch("switch1", cb)

await bus.emit(topic='?act', uid=sw1.uid('state'), value='on')
await bus.emit(topic='?act', uid=sw1.uid('state'), value='off')

EVENT !state: {'uid': 'test.switch1.state', 'value': 'ON', 'timestamp': 1723506587.114561, 'src': 'test'}
EVENT !state: {'uid': 'test.switch1.state', 'value': 'OFF', 'timestamp': 1723506587.114561, 'src': 'test'}


## Light

In [6]:
from eventbus import bus, Light

async def cb(light: Light, uid, value):
    # set light to requested state ..., then:
    await light.update(uid, value)

light1 = Light("light1", cb)


await bus.emit(topic='?act', uid=light1.uid('state'), value='on')
await bus.emit(topic='?act', uid=light1.uid('brightness'), value=55)


EVENT !state: {'uid': 'test.light1.state', 'value': 'ON', 'timestamp': 1723506587.114561, 'src': 'test'}
EVENT !state: {'uid': 'test.light1.brightness', 'value': 55, 'timestamp': 1723506587.114561, 'src': 'test'}
